In [1]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier

In [2]:
X = np.load('day7_data2_X.npy')
y = np.load('day7_data2_y.npy')

In [3]:
c1 = LogisticRegression(random_state=1)
c2 = DecisionTreeClassifier(random_state=1, max_depth=4)
c3 = GaussianNB()
ec = VotingClassifier(estimators=[('lr',c1),('rf',c2),('gnb',c3)],voting='hard')

In [4]:
from sklearn.model_selection import cross_val_score
cross_val_score(ec, X, y, cv=5).mean()

0.8222941661905668

In [5]:
cross_val_score(c1, X, y, cv=5).mean()

0.8290420872214816

In [6]:
cross_val_score(c2, X, y, cv=5).mean()

0.8223068621849807

In [7]:
cross_val_score(c3, X, y, cv=5).mean()

0.4600139655938551

In [8]:
c1 = LogisticRegression(random_state=1)
c2 = DecisionTreeClassifier(random_state=1)
ec1 = VotingClassifier(estimators=[('lr',c1),('dt',c2)],voting='hard')
cross_val_score(ec1, X, y, cv=5).mean()

0.8222687742017394

In [9]:
#LogisticRegression(C=,penalty=,solver=,)
#DecisionTreeClassifier(min_samples_leaf=max_depth=criterion=)
c_params = [0.1, 5.0, 7.0, 10.0, 15.0, 20.0, 100.0]

params = {
    'lr__solver':['liblinear'], # extend라 구조로 전달
    'lr__penalty':['l2'],
    'lr__C':c_params,
    'dt__criterion':['gini', 'entropy'],
    'dt__max_depth':[10,8,7,6,5,4,3,2],
    'dt__min_samples_leaf':[1,2,3,4,5,6,7,8,9],
}

In [10]:
from sklearn.model_selection import GridSearchCV
g = GridSearchCV(estimator=ec1, param_grid=params, cv=5)
g.fit(X,y)
g.best_score_ # 다소 시간이 걸린다.

0.8425569732749316

In [11]:
g.best_params_

{'dt__criterion': 'gini',
 'dt__max_depth': 10,
 'dt__min_samples_leaf': 5,
 'lr__C': 5.0,
 'lr__penalty': 'l2',
 'lr__solver': 'liblinear'}

In [12]:
ec2 = VotingClassifier(estimators=[('A',c1),('B',c2)],voting='hard')
params = {'B__criterion': ['gini'],
 'B__max_depth': [10],
 'B__min_samples_leaf': [5],
 'A__C': [5.0],
 'A__penalty': ['l2'],
 'A__solver': ['liblinear']}
g1 = GridSearchCV(estimator=ec2, param_grid=params, cv=5)
#g.best_score_ 단일이라 안나옴.

In [13]:
c1 = LogisticRegression(solver='liblinear',penalty='l2',C=5.0, random_state=1)
c2 = DecisionTreeClassifier(criterion='gini',max_depth=10,min_samples_leaf=5,random_state=1)

ec3 = VotingClassifier(estimators=[('A',c1),('B',c2)],voting='hard')
cross_val_score(ec3,X,y,cv=5).mean()

0.8425569732749316

앙상블에서 그냥  best score를 적용하는 것 보다 best parameter를 찾아서 모델에 직접 입력해준다음 교차검증 하는 것이 성능적인 측면에서 뛰어나다.

정리 조합

In [14]:
import numpy as np # 파일로드
from sklearn.linear_model import LogisticRegression # 모델1
from sklearn.tree import DecisionTreeClassifier # 모델2
from sklearn.ensemble import VotingClassifier #모델조합
from sklearn.model_selection import cross_val_score # 성능평가

# data로드
X = np.load('day7_data2_X.npy')
y = np.load('day7_data2_y.npy')
# 모델생성(최적의 파라미터)
c1 = LogisticRegression(solver='liblinear',penalty='l2',C=5.0, random_state=1)
c2 = DecisionTreeClassifier(criterion='gini',max_depth=10,min_samples_leaf=5,random_state=1)
# 앙상블 모델 생성
ec3 = VotingClassifier(estimators=[('c1',c1),('c2',c2)],voting='hard')
# 검증
cross_val_score(ec3,X,y,cv=5).mean()

0.8425569732749316